In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
# @save
# d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
#                                 '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# text lines: {len(lines)}')
print(lines[0])
print(lines[10])

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


Tokenization

In [3]:
def tokenize(lines, token='word'):
    '''
    split text lines into word or character tokens.
    '''
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else :
        print('ERROR : unknown token type: '+ token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


Property function

In [4]:
class Person:
    def __init__(self,firstname, lastname, age):
        self.firstname = firstname
        self.lastname = lastname
        self.age = age
    
    def get_age(self):
        return self._age
    
    def set_age(self, age):
        if age < 0 :
            raise ValueError("Invalid age")
        self._age = age
    age = property(get_age, set_age)      # getter setter 호출

It is possilbe to simplify this

In [5]:
class Person:
    def __init__(self,firstname, lastname, age):
        self.firstname = firstname
        self.lastname = lastname
        self.age = age
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if age < 0 :
            raise ValueError("Invalid age")
        self._age = age

me = Person('wonseok', 'jang', 10)
me.age = -11
me.age

ValueError: Invalid age

Another example

In [6]:
class Person:
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age
    
    # @property
    def full_name(self):
        return self.first_name + " " + self.last_name

me = Person('wonseok','jang', 30)
print(me.full_name())


class Person:
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age
    
    @property
    def full_name(self):
        return self.first_name + " " + self.last_name

me = Person('wonseok','jang', 30)
print(me.full_name)

wonseok jang
wonseok jang


In [7]:
class Vocab:
    '''
    Vocabulary for text
    '''
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None :
            reserved_tokens = []
        # Sort according to frequencies
        counter = count_corpus(tokens)
        # sorting by keys
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        # The index for unknwon token is 0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token : idx for idx, token in enumerate(self.idx_to_token)}
        
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) -1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    @property
    def unk(self): # index for the unknown token
        return 0
    
    @property
    def token_freqs(self): # Index for the unkown token
        return self._token_freqs
    
def count_corpus(tokens:list):
    '''
    Count token frequencies
    '''
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    
    return collections.Counter(tokens)
            

In [8]:
vocab = Vocab(tokens)

In [10]:
for i in [0,10]:
    print('words', tokens[i])
    print('indices' , vocab[tokens[i]])

words ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices [1, 19, 50, 40, 2183, 2184, 400]
words ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [20]:
def load_corpus_time_machine(max_tokens=-1):
    '''
    returns token indices and the vocabulary of the time machine dataset.
    '''
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # Since each text line in the time machiine dataset is not necessarily a 
    # sentence or a paragraph, flatten all the text lines into a single list
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0 :
        corpus = corpus[:max_tokens]
    return corpus, vocab


_, voc = load_corpus_time_machine()